In [9]:
import sqlite3
import pandas as pd
from datetime import datetime

In [10]:
def insert_unique_data_to_db(df, table_name, conn, cursor):
    """
    Wstawia unikalne dane do bazy danych, sprawdzając, czy rekord już istnieje w tabeli.

    Args:
        df (DataFrame): DataFrame z danymi do wstawienia.
        table_name (str): Nazwa tabeli, do której dane mają być wstawione.
        conn (sqlite3.Connection): Obiekt połączenia z bazą danych SQLite.
        cursor (sqlite3.Cursor): Obiekt kursora SQLite.
    """
    df.columns = df.columns.str.replace(' ', '_').str.replace('ł', 'l').str.replace('ż', 'z')
    df['data'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    for _, row in df.iterrows():
        query_check = f'''
        SELECT 1 FROM {table_name}
        WHERE Cena = ? AND Cena_za_m2 = ? AND Lokalizacja = ? AND URL = ?
        '''
        cursor.execute(query_check, (row['Cena'], row['Cena_za_m2'], row['Lokalizacja'], row['URL']))
        exists = cursor.fetchone()

        if not exists:  
            query_insert = f'''
            INSERT INTO {table_name} (
                Cena, Cena_za_m2, Lokalizacja, Powierzchnia_calkowita, Liczba_pokoi, Piętro, Typ_budynku, Rok_budowy,
                Typ_ogloszeniodawcy, URL, Forma_wlasnosci, Dostepne_od, Stan_wykonczenia, Rynek, winda, ogrzewanie,
                wojewodztwo, miasto, dzielnica, ulica, Liczba_pieter, balkon, piwnica, garaz, miejsce_parkingowe,
                Ogrzewanie_rodzaj, ochrona, Ogłoszeniodawca, dzielnica_num, data
            ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);
            '''
            cursor.execute(query_insert, tuple(row))
    conn.commit()
    print(f"Dane zostały wstawione do tabeli '{table_name}'.")



In [11]:
import sqlite3
import pandas as pd
from datetime import datetime

#df = pd.read_excel(r'C:\Users\PC\Desktop\scrapping_danych\Data_all_preprocessed_actual.xlsx') 
df = pd.read_excel(r'C:\Users\PC\Desktop\scrapping_danych\Data_all_preprocessed_actual.xlsx') 
df_arch = pd.read_excel(rf'C:\Users\PC\Desktop\scrapping_danych\Data_all_preprocessed_.xlsx')  

df['Cena'] = df['Cena'].astype(float)
df['Cena za m2'] = df['Cena za m2'].astype(float)
df['Powierzchnia całkowita'] = df['Powierzchnia całkowita'].astype(float)
df['Liczba pokoi'] = df['Liczba pokoi'].astype(int)
df['Liczba pięter'] = pd.to_numeric(df['Liczba pięter'], errors='coerce') 
df['Piętro'] = pd.to_numeric(df['Piętro'], errors='coerce')


df.fillna('brak informacji', inplace=True)

df_arch['Cena'] = df_arch['Cena'].astype(float)
df_arch['Cena za m2'] = df_arch['Cena za m2'].astype(float)
df['Powierzchnia całkowita'] = df['Powierzchnia całkowita'].astype(float)
df_arch['Liczba pokoi'] = df_arch['Liczba pokoi'].astype(int)
df_arch['Liczba pięter'] = pd.to_numeric(df_arch['Liczba pięter'], errors='coerce') 
df_arch['Piętro'] = pd.to_numeric(df_arch['Piętro'], errors='coerce')


df_arch.fillna('brak informacji', inplace=True)
conn = sqlite3.connect('real_estate2.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS aktualne (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Cena FLOAT,
    Cena_za_m2 FLOAT,
    Lokalizacja TEXT,
    Powierzchnia_calkowita FLOAT,
    Liczba_pokoi INTEGER,
    Piętro FLOAT,
    Typ_budynku TEXT,
    Rok_budowy FLOAT,
    Typ_ogloszeniodawcy TEXT,
    URL TEXT,
    Forma_wlasnosci TEXT,
    Dostepne_od TEXT,
    Stan_wykonczenia TEXT,
    Rynek TEXT,
    winda INTEGER,
    ogrzewanie INTEGER,
    wojewodztwo TEXT,
    miasto TEXT,
    dzielnica TEXT,
    ulica TEXT,
    Liczba_pieter FLOAT,
    balkon INTEGER,
    piwnica INTEGER,
    garaz INTEGER,
    miejsce_parkingowe INTEGER,
    Ogrzewanie_rodzaj TEXT,
    ochrona INTEGER,
    Ogłoszeniodawca TEXT,
    dzielnica_num INTEGER,
    data TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS archiwalne (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Cena FLOAT,
    Cena_za_m2 FLOAT,
    Lokalizacja TEXT,
    Powierzchnia_calkowita FLOAT,
    Liczba_pokoi INTEGER,
    Piętro FLOAT,
    Typ_budynku TEXT,
    Rok_budowy FLOAT,
    Typ_ogloszeniodawcy TEXT,
    URL TEXT,
    Forma_wlasnosci TEXT,
    Dostepne_od TEXT,
    Stan_wykonczenia TEXT,
    Rynek TEXT,
    winda INTEGER,
    ogrzewanie INTEGER,
    wojewodztwo TEXT,
    miasto TEXT,
    dzielnica TEXT,
    ulica TEXT,
    Liczba_pieter FLOAT,
    balkon INTEGER,
    piwnica INTEGER,
    garaz INTEGER,
    miejsce_parkingowe INTEGER,
    Ogrzewanie_rodzaj TEXT,
    ochrona INTEGER,
    Ogłoszeniodawca TEXT,
    dzielnica_num INTEGER,
    data TEXT
)
''')

def insert_data_to_db(df, table_name):
    df['data'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    records = df.to_records(index=False)
    data = [tuple(record) for record in records]

    query = f'''
    INSERT INTO {table_name} (
        Cena, Cena_za_m2, Lokalizacja, Powierzchnia_calkowita, Liczba_pokoi, Piętro, Typ_budynku, Rok_budowy,
        Typ_ogloszeniodawcy, URL, Forma_wlasnosci, Dostepne_od, Stan_wykonczenia, Rynek, winda, ogrzewanie, wojewodztwo,
        miasto, dzielnica, ulica, Liczba_pieter, balkon, piwnica, garaz, miejsce_parkingowe, Ogrzewanie_rodzaj,
        ochrona, Ogłoszeniodawca, dzielnica_num, data
    ) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);
    '''
    cursor.executemany(query, data)
    conn.commit()

def clear_table(table_name):
    query = f"DELETE FROM {table_name};"
    cursor.execute(query)
    conn.commit()
    print(f"Zawartość tabeli '{table_name}' została usunięta.")

clear_table('aktualne')
df.columns = df.columns.str.replace(' ', '_').str.replace('ł', 'l').str.replace('ż', 'z')
df.to_sql('aktualne', conn, if_exists='replace', index=False)
insert_unique_data_to_db(df, 'archiwalne', conn, cursor)
#df_arch.columns = df_arch.columns.str.replace(' ', '_').str.replace('ł', 'l').str.replace('ż', 'z')
#df_arch.to_sql('archiwalne', conn, if_exists='replace', index=False)
#insert_data_to_db(df, 'aktualne')
# insert_data_to_db(df_archiwalne, 'archiwalne')
conn.close()

print("Dane zapisane do bazy.")


C:\Users\PC\AppData\Local\Temp\ipykernel_8004\4285524810.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'brak informacji' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('brak informacji', inplace=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_8004\4285524810.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'brak informacji' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_arch.fillna('brak informacji', inplace=True)


Zawartość tabeli 'aktualne' została usunięta.
Dane zostały wstawione do tabeli 'archiwalne'.
Dane zapisane do bazy.


In [12]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('real_estate2.db')
query = """SELECT * from archiwalne
"""
df_from_db = pd.read_sql(query, conn)
conn.close()

df_from_db

,id,Cena,Cena_za_m2,Lokalizacja,Powierzchnia_calkowita,Liczba_pokoi,Piętro,Typ_budynku,Rok_budowy,Typ_ogloszeniodawcy,...,Liczba_pieter,balkon,piwnica,garaz,miejsce_parkingowe,Ogrzewanie_rodzaj,ochrona,Ogłoszeniodawca,dzielnica_num,data
0,1,440000.0,16865.00,"ul. Nadgórników, Bogucice, Katowice, śląskie",26.09,1,8.0,blok,2024.0,biuro nieruchomości,...,11.0,1,0,0,0,brak informacji,1,brak informacji,1,2025-01-12 19:05:50
1,2,385000.0,10405.00,"ul. Strzelecka, Śródmieście, Katowice, śląskie",37.00,2,1.0,kamienica,brak informacji,biuro nieruchomości,...,brak informacji,0,1,0,0,miejskie,0,brak informacji,1,2025-01-12 19:05:50
2,3,1092625.0,12345.00,"Olimpijska/al. Korfantego, Bogucice, Katowice,...",88.51,4,10.0,brak informacji,2026.0,deweloper,...,18.0,0,0,1,1,brak informacji,0,brak informacji,1,2025-01-12 19:05:50
3,4,490000.0,8886.00,"ul. 18 Sierpnia, Załęże, Katowice, śląskie",55.14,2,0.0,blok,1960.0,prywatny,...,3.0,0,1,0,0,miejskie,0,brak informacji,2,2025-01-12 19:05:50
4,5,632723.0,11900.00,"ul. Złota, Dąb, Katowice, śląskie",53.17,3,1.0,brak informacji,2024.0,deweloper,...,4.0,1,0,1,1,brak informacji,1,brak informacji,3,2025-01-12 19:05:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5499,5500,649000.0,10417.34,"śląskie,Katowice,Os. Tysiąclecia,Zawiszy Czarnego",62.30,3,6.0,blok,1991.0,brak informacji,...,25.0,1,0,0,0,brak informacji,0,Kawka Nieruchomości,2,2025-01-14 17:57:00
5500,5501,364027.0,13799.36,"śląskie,Katowice,Księdza Piotra Skargi",26.38,1,5.0,inny,2026.0,brak informacji,...,11.0,1,0,0,0,brak informacji,1,HOMEPLUS Nieruchomości,1,2025-01-14 17:57:00
5501,5502,439000.0,14972.71,"śląskie,Katowice,Centrum,Nadgórników",29.32,1,3.0,park,brak informacji,brak informacji,...,10.0,1,0,0,0,brak informacji,0,CARBON ESTATE,1,2025-01-14 17:57:00
5502,5503,108000.0,4695.65,"śląskie,Katowice,Dąbrówka Mała,Hubala Dobrzańs...",23.00,1,2.0,inny,brak informacji,brak informacji,...,3.0,0,0,0,0,brak informacji,0,WGN NIERUCHOMOŚCI / Wrocław,3,2025-01-14 17:57:00


In [15]:
df_arch[df_from_db.duplicated(subset=['URL']) == 'True']

C:\Users\PC\AppData\Local\Temp\ipykernel_8004\3133066805.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_arch[df_from_db.duplicated(subset=['URL']) == 'True']


,ogrzewanie,Piętro,Stan wykończenia,Rynek,Forma własności,Dostępne od,Typ ogłoszeniodawcy,Rok budowy,winda,Typ budynku,...,ulica,Liczba pięter,balkon,piwnica,garaż,miejsce parkingowe,Ogrzewanie_rodzaj,ochrona,Ogłoszeniodawca,dzielnica_num


In [41]:
import sqlite3
import requests
import pandas as pd
from bs4 import BeautifulSoup

def check_page_status(url):
    """Sprawdza status strony i obecność komunikatu o nieaktualności."""
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        if response.status_code != 200:
            print(f"Strona niedostępna: {url}")
            return False
        soup = BeautifulSoup(response.text, 'html.parser')

        domiporta_message = soup.find('div', class_='archive__info')
        if domiporta_message and soup.find('h3', class_='archive__title'):
            print(f"Ogłoszenie nieaktualne w Domiporta: {url}") 
            return False
        otodom_message = soup.find('div', {'data-cy': 'expired-ad-alert'})
        if otodom_message and soup.find('span', class_='css-5ujp2z'):
            print(f"Ogłoszenie nieaktualne w Otodom: {url}")
            return False

        print(f"Ogłoszenie aktualne: {url}")
        return True

    except Exception as e:
        print(f"Błąd podczas sprawdzania strony {url}: {e}")
        return False


In [24]:
'''
df_from_db['URL_exists'] = df_from_db['URL'].apply(check_page_status)
non_existent_urls = df_from_db[df_from_db['URL_exists'] == False]
print("URL, które nie istnieją:")
print(non_existent_urls[['id', 'URL']])
'''


Strona niedostępna: https://www.otodom.pl/pl/oferta/centrum-katowice-zabrska-mickiewicza-22-pietro-ID4tbuR
Ogłoszenie aktualne: https://www.otodom.pl/pl/oferta/medykow-32-mieszkanie-3-pok-84-ID4nj2X
Ogłoszenie aktualne: https://www.domiporta.pl/nieruchomosci/sprzedam-mieszkanie-trzypokojowe-katowice-koszutka-jesionowa-94m2/154975177
Ogłoszenie aktualne: https://www.domiporta.pl/nieruchomosci/sprzedam-mieszkanie-dwupokojowe-katowice-brynow-mikolowska-50m2/155227276
Ogłoszenie aktualne: https://www.otodom.pl/pl/oferta/przestronne-m360-21m2burowiec-ID4tt0R
Strona niedostępna: https://www.otodom.pl/pl/oferta/mieszkanie-50-55-m-katowice-ID4t81T
Ogłoszenie aktualne: https://www.otodom.pl/pl/oferta/panorama-3-stawy-mieszkanie-2-pok-4-b34-ID4nVCG
Ogłoszenie aktualne: https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-39m2-loggia-ID4pFtn
Ogłoszenie aktualne: https://www.otodom.pl/pl/oferta/medykow-32-mieszkanie-4-pok-55-ID4nj2n
Ogłoszenie nieaktualne w Domiporta: https://www.domiporta.pl/nie

In [ ]:
'''
ids_to_remove = non_existent_urls['id'].tolist()

if ids_to_remove:
    placeholders = ', '.join('?' for _ in ids_to_remove)
    query = f"DELETE FROM your_table_name WHERE id IN ({placeholders})"
    cursor.execute(query, ids_to_remove)
    conn.commit()
    print(f"Usunięto {cursor.rowcount} rekordów z bazy danych.")
else:
    print("Brak nieistniejących URL do usunięcia.")
'''